## Speed Calculator
This file assists in calculating switch times/speeds for toggle switch models. Requires the dynaCA package developed in LMSE by Raj K. (also available on GitHub) in order to work.

In [1]:
from dynaCA.core import DynaModel, SwitchModel
from dynaCA.plotting import *
from dynaCA.core.DynaTrajectory import *
from scipy.integrate import solve_ivp
from numpy.linalg import det
import pickle
from joblib import Parallel, delayed
import multiprocessing
import time

# Define functions
Speed of switching calculated at kp values defined by 4 corners of robustness landscape

In [2]:
def simple_toggle_ode(concs, params):
    lac_i, tet_r = concs
    
    kp_1 = params['kp_1']
    kp_2 = params['kp_2']
    kdiss_1 = params['kdiss_1']
    kdiss_2 = params['kdiss_2']
    kdeg_1 = params['kdeg_1']
    kdeg_2 = params['kdeg_2']
    n_1 = params['n_1']
    n_2 = params['n_2']
    
    dlaci_dt = kp_1/(1+(tet_r/kdiss_1)**n_1) - kdeg_1*lac_i
    dtetr_dt = kp_2/(1+(lac_i/kdiss_2)**n_2) - kdeg_2*tet_r

    return [dlaci_dt, dtetr_dt]    



def simple_toggle_jacobian(concs, params):
    lac_i, tet_r = concs

    kp_1 = params['kp_1']
    kp_2 = params['kp_2']
    kdiss_1 = params['kdiss_1']
    kdiss_2 = params['kdiss_2']
    kdeg_1 = params['kdeg_1']
    kdeg_2 = params['kdeg_2']
    n_1 = params['n_1']
    n_2 = params['n_2']

    return [[-kdeg_1, -(kp_1*n_1/kdiss_1**n_1)*(tet_r**(n_1-1)/(1+(tet_r/kdiss_1)**n_1)**2)],
            [-(kp_2*n_2/kdiss_2**n_2)*(lac_i**(n_2-1)/(1+(lac_i/kdiss_2)**n_2)**2),  -kdeg_2 ]]
        
    
    
def simple_toggle_nullclines(conc_ranges, params):
    lac_range, tet_range = conc_ranges
    
    kp_1 = params['kp_1']
    kp_2 = params['kp_2']
    kdiss_1 = params['kdiss_1']
    kdiss_2 = params['kdiss_2']
    kdeg_1 = params['kdeg_1']
    kdeg_2 = params['kdeg_2']
    n_1 = params['n_1']
    n_2 = params['n_2']
    lac_null = kp_1/(kdeg_1*(1+(tet_range/kdiss_1)**n_1))
    tet_null = kp_2/(kdeg_2*(1+(lac_range/kdiss_2)**n_2))
    
    return lac_null, tet_null

def plot_robustness_landscape(results):
    z = [result['robustness']/1e6 for result in results]
    param_space = np.logspace(-3,3,30)
    trace_list = []
    trace_list.append(go.Contour(x=param_space, y=param_space,
                                 z=np.array(z).reshape(30,30).transpose(), 
                                 colorscale='RdBu', zmin=0, zmax=1, ncontours=12,
                                 contours=dict(showlabels=True, coloring='heatmap', labelfont=dict(family='Myriad Pro', size=14, color='black')),
                                 colorbar=dict(title='Fraction of Production<br>Parametric Space Bistable', titleside='right', titlefont=dict(family='Myriad Pro', size=18, color='black'),
                                               thickness=16, len=1.05, ticks='outside', tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black')))
    layout = go.Layout(height=500, width=525,

                       xaxis=dict(title='kdeg_1',
                                  titlefont=dict(family='Myriad Pro', size=18, color='black'), title_standoff=0.2,
                                  showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                                  ticks='outside', ticklen=4, tickangle=-90, nticks=12, type='log',
                                  tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                                  showgrid=True,zeroline=False,range=(-3,3)),

                      yaxis=dict(title='kdeg_2',
                                titlefont=dict(family='Myriad Pro', size=18, color='black'),title_standoff=0.2,

                                  anchor='x', side='left', showgrid=True, zeroline=False, type='log',
                                tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                                  range=(-3, 3),showline=True, linewidth=1, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=4, tickangle=0))

    fig = go.Figure(data=trace_list, layout=layout)
    return fig

def get_result_from_list(results_list, kdeg_1=0.001, kdeg_2=0.001, mode='closest'):
    #param_space = np.linspace(np.logspace(-3,3,30)[-7], np.logspace(-3,3,30)[-2], 30)
    param_space = np.logspace(-3,3,30)
    #param_space = np.append(np.geomspace(0.1,92,16), (481.5+92-(np.geomspace(481.5,92,15))[1:]))
    if mode=='closest':
        kdeg_1 = param_space[np.argmin(abs(param_space-kdeg_1))]
        kdeg_2 = param_space[np.argmin(abs(param_space-kdeg_2))]
    elif mode=='less':
        try:
            kdeg_1 = param_space[param_space-kdeg_1<0][-1]
        except IndexError:
            kdeg_1 = min(param_space)
        try:
            kdeg_2 = param_space[param_space-kdeg_2<0][-1]
        except IndexError:
            kdeg_2 = min(param_space)

    elif mode=='greater':
        try:
            kdeg_1 = param_space[param_space-kdeg_1>0][0]
        except IndexError:
            kdeg_1 = max(param_space)
        try:
            kdeg_2 = param_space[param_space-kdeg_2>0][0]
        except IndexError:
            kdeg_2 = max(param_space)
    temp = [result for result in results_list if result['params']['kdeg_1']==kdeg_1 and result['params']['kdeg_2']==kdeg_2]
    if len(temp)>1:
        raise Exception('Duplicate entries found. Please check.')
    elif len(temp)==0:
        raise Exception('No matching record found.')
    else:
        return dict(temp[0])

In [3]:
def obtain_switch_times(robustness_results, kdeg_1=1, kdeg_2=1):
    result = get_result_from_list(robustness_results, kdeg_1, kdeg_2, mode='closest')
    params = dict(result['params'])
    del params['kp_1']
    del params['kp_2']
    func_dict = {'bottom_corner': get_min_corner_kp,
                 'top_left_corner': get_upper_corner_kp,
                 'top_right_corner': get_lower_corner_kp,
                 'top_corner': get_corner_kp}
    switch_times = {'params': params,
                    'bottom_corner': {},
                    'top_left_corner': {},
                    'top_corner': {},
                    'top_right_corner': {}}
    temp_params = dict(params)
    for point in ['bottom_corner', 'top_left_corner', 'top_right_corner', 'top_corner']:
        if len(result['lower_x']) > 5 and len(result['upper_x']) > 5:
            temp_params['kp_1'], temp_params['kp_2'] = func_dict[point](result)
        else:
            temp_params['kp_1'], temp_params['kp_2'] = None, None
        if result['robustness']>0 and len(result['lower_x']) > 5 and len(result['upper_x']) > 5:
            if temp_params['kp_1'] and temp_params['kp_2']:
                if temp_params['kp_1']>0 and temp_params['kp_2']>0:
                    switch_times[point]['kp_1'], switch_times[point]['kp_2'] = temp_params['kp_1'], temp_params['kp_2']
                    try:
                        switch_model = SwitchModel(num_state_variables = 2, state_variables =['lac', 'tet'],
                                                   model_params=temp_params, model_eqns=simple_toggle_ode,
                                                  jacobian_eqns=simple_toggle_jacobian,
                                                  nullcline_eqns=simple_toggle_nullclines, verbose=True, fast_calculate=True)
                    except:
                        switch_model = None
                    if switch_model:
                        if len(switch_model.steady_states['stable'])>=2 and len(switch_model.steady_states['unstable'])>=1:
                            switch_times[point]['switch_times'] = switch_model.switch_times
                        else:
                            switch_times[point]['switch_times'] = None
                    else:
                        switch_times[point]['switch_times'] = None
                else:
                    switch_times[point]['kp_1'], switch_times[point]['kp_2'] = np.nan, np.nan
                    switch_times[point]['switch_times'] = None
            else:
                switch_times[point]['kp_1'], switch_times[point]['kp_2'] = np.nan, np.nan
                switch_times[point]['switch_times'] = None
        else:
            switch_times[point]['kp_1'], switch_times[point]['kp_2'] = np.nan, np.nan
            switch_times[point]['switch_times'] = None
    del result
    return switch_times

def obtain_switch_times_final(robustness_results, kdeg_1=1, kdeg_2=1, plus=3):
    result = get_result_from_list(robustness_results, kdeg_1, kdeg_2, mode='closest')
    params = dict(result['params'])
    del params['kp_1']
    del params['kp_2']
    func_dict = {'bottom_corner': get_min_corner_kp_final,
                 'top_left_corner': get_upper_corner_kp,
                 'top_right_corner': get_lower_corner_kp,
                 'top_corner': get_corner_kp}
    switch_times = {'params': params,
                    'bottom_corner': {},
                    'top_left_corner': {},
                    'top_corner': {},
                    'top_right_corner': {}}
    temp_params = dict(params)
    for point in ['bottom_corner', 'top_left_corner', 'top_right_corner', 'top_corner']:
        if len(result['lower_x']) > 5 and len(result['upper_x']) > 5:
            if point=='bottom_corner':
                temp_params['kp_1'], temp_params['kp_2'] = func_dict[point](result, plus)
            else:
                temp_params['kp_1'], temp_params['kp_2'] = func_dict[point](result)
        else:
            temp_params['kp_1'], temp_params['kp_2'] = None, None
        if result['robustness']>0 and len(result['lower_x']) > 5 and len(result['upper_x']) > 5:
            if temp_params['kp_1'] and temp_params['kp_2']:
                if temp_params['kp_1']>0 and temp_params['kp_2']>0:
                    switch_times[point]['kp_1'], switch_times[point]['kp_2'] = temp_params['kp_1'], temp_params['kp_2']
                    try:
                        switch_model = SwitchModel(num_state_variables = 2, state_variables =['lac', 'tet'],
                                                   model_params=temp_params, model_eqns=simple_toggle_ode,
                                                   jacobian_eqns=simple_toggle_jacobian,
                                                   nullcline_eqns=simple_toggle_nullclines)
                    except:
                        switch_model = None
                    if switch_model:
                        if len(switch_model.steady_states['stable'])>=2 and len(switch_model.steady_states['unstable'])>=1:
                            switch_times[point]['switch_times'] = switch_model.switch_times
                        else:
                            switch_times[point]['switch_times'] = None
                    else:
                        switch_times[point]['switch_times'] = None
                else:
                    switch_times[point]['kp_1'], switch_times[point]['kp_2'] = np.nan, np.nan
                    switch_times[point]['switch_times'] = None
            else:
                switch_times[point]['kp_1'], switch_times[point]['kp_2'] = np.nan, np.nan
                switch_times[point]['switch_times'] = None
        else:
            switch_times[point]['kp_1'], switch_times[point]['kp_2'] = np.nan, np.nan
            switch_times[point]['switch_times'] = None
    del result
    return switch_times


def get_min_corner_kp(result):
    if result['lower_x'][1] == result['upper_x'][1]:
        if any(np.where(result['upper_y'][:1000]<result['lower_y'][:1000])[0]):
            temp_min_index = np.where(result['upper_y'][:1000]<result['lower_y'][:1000])[0][-1] + 1
        else:
            temp_min_index = 0
        try:
            min_index = temp_min_index + np.argmin(result['upper_y'][temp_min_index:1000] - result['lower_y'][temp_min_index:1000])
        except:
            min_index = temp_min_index
        if min_index ==1000:
            min_index = 998
        return  result['lower_x'][min_index+1], result['lower_y'][min_index+1] + (result['upper_y'][min_index+1]-result['lower_y'][min_index+1])/2
    elif result['lower_y'][1] == result['upper_y'][1]:
        if any(np.where(result['lower_x'][:1000]<result['upper_x'][:1000])[0]):
            temp_min_index = np.where(result['lower_x'][:1000]<result['upper_x'][:1000])[0][-1] + 1
        else:
            temp_min_index = 0
        try:
            min_index = temp_min_index + np.argmin(result['lower_x'][temp_min_index:1000] - result['upper_x'][temp_min_index:1000])
        except:
            min_index = temp_min_index
        if min_index == 1000:
            min_index = 998
        return result['upper_x'][min_index+1] + (result['lower_x'][min_index+1] - result['upper_x'][min_index+1])/2, result['lower_y'][min_index+1]
    else:
        upper_slope = (result['upper_y'][1] - result['upper_y'][0])/(result['upper_x'][1] - result['upper_x'][0])
        lower_slope = (result['lower_y'][1] - result['lower_y'][0])/(result['lower_x'][1] - result['lower_x'][0])
        m = (upper_slope + lower_slope)/2
        x = result['lower_x'][0] + 1/np.sqrt(m**2+1)
        y = result['lower_y'][0] + m*1/np.sqrt(m**2+1)
        return x,y
    
def get_min_corner_kp_final(result, plus=3):
    if result['lower_x'][1] == result['upper_x'][1]:
        if any(np.where(result['upper_y'][:1000]<result['lower_y'][:1000])[0]):
            temp_min_index = np.where(result['upper_y'][:1000]<result['lower_y'][:1000])[0][-1] + 1
        else:
            temp_min_index = 0
        try:
            min_index = temp_min_index + np.argmin(result['upper_y'][temp_min_index:1000] - result['lower_y'][temp_min_index:1000])
        except:
            min_index = temp_min_index
        if min_index == 1000:
            min_index = 998
        return  result['lower_x'][min_index+plus], result['lower_y'][min_index+plus] + (result['upper_y'][min_index+plus]-result['lower_y'][min_index+plus])/2
    elif result['lower_y'][1] == result['upper_y'][1]:
        if any(np.where(result['lower_x'][:1000]<result['upper_x'][:1000])[0]):
            temp_min_index = np.where(result['lower_x'][:1000]<result['upper_x'][:1000])[0][-1] + 1
        else:
            temp_min_index = 0
        try:
            min_index = temp_min_index + np.argmin(result['lower_x'][temp_min_index:1000] - result['upper_x'][temp_min_index:1000])
        except:
            min_index = temp_min_index
        if min_index == 1000:
            min_index = 999-plus
        return result['upper_x'][min_index+plus] + (result['lower_x'][min_index+plus] - result['upper_x'][min_index+plus])/2, result['lower_y'][min_index+plus]
    else:
        upper_slope = (result['upper_y'][1] - result['upper_y'][0])/(result['upper_x'][1] - result['upper_x'][0])
        lower_slope = (result['lower_y'][1] - result['lower_y'][0])/(result['lower_x'][1] - result['lower_x'][0])
        m = (upper_slope + lower_slope)/2
        x = result['lower_x'][0] + (plus*2)/np.sqrt(m**2+1)
        y = result['lower_y'][0] + m*(plus*2)/np.sqrt(m**2+1)
        return x,y

    
def get_upper_corner_kp(result):
    if np.argmax(result['upper_y']==1000)!=0:
        upper_curve_corner_index = np.argmax(result['upper_y']==1000)+1
        if upper_curve_corner_index+1 >= len(result['upper_y']):
            upper_curve_corner_index = upper_curve_corner_index - 1
        uc_kp_1 = result['upper_x'][upper_curve_corner_index] + result['upper_x'][upper_curve_corner_index]/1000
        uc_kp_2 = result['upper_y'][upper_curve_corner_index]
    else:
        uc_kp_1 = result['upper_x'][-1]
        uc_kp_2 = result['upper_y'][-1]-result['upper_y'][-1]/1000
    return uc_kp_1, uc_kp_2

def get_lower_corner_kp(result):
    if np.argmax(result['lower_y']==1000)==0:

        lc_kp_1 = result['lower_x'][-1]
        lc_kp_2 = result['lower_y'][-1] + result['lower_y'][-1]/1000
    else:
        lc_kp_1 = result['lower_x'][-1] - result['lower_x'][-1]/1000
        lc_kp_2 = result['upper_y'][-1]
    return lc_kp_1, lc_kp_2
    
def get_corner_kp(result):
    lc_kp_1, lc_kp_2 = get_lower_corner_kp(result)
    uc_kp_1, uc_kp_2 = get_upper_corner_kp(result)
    
    if lc_kp_1==1000 and uc_kp_2==1000:
        return 1000,1000
    else:
        return (lc_kp_1+uc_kp_1)/2, (lc_kp_2+uc_kp_2)/2
def plot_robustness(results):
    trace_list = []
    for i, result in enumerate(results):
        trace_list.append(go.Scatter(x=result['lower_x'], y=result['lower_y'], line=dict(color=get_line_color(2)[i]), name='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4)), 
                                     legendgroup='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4))))
        trace_list.append(go.Scatter(x=result['upper_x'], y=result['upper_y'], fill='tonexty', line=dict(color=get_line_color(2)[i]), name='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4)),
                                     legendgroup='kdeg_1='+str(np.round(result['params']['kdeg_1'], decimals=4))+', kdeg_2='+str(np.round(result['params']['kdeg_2'], decimals=4)), showlegend=False))


    layout = go.Layout(height=500, width=675, legend_orientation='v', legend_x = 1.05, legend_y = .95, legend_font = dict(family='Myriad Pro', size=14, color='black'),xaxis=dict(title='kp_1',
                      titlefont=dict(family='Myriad Pro', size=18, color='black'), title_standoff=0,
                      showline=True, linewidth=1, linecolor='black', mirror=True, side='bottom',
                      ticks='outside', ticklen=4, tickangle=0, nticks=8, 
                      tickfont=dict(size=16, family='Myriad Pro', color='black'), tickcolor='black',
                      showgrid=True,zeroline=False,range=(0,1000)),yaxis=dict(title='kp_2',
                                titlefont=dict(family='Myriad Pro', size=18, color='black'),title_standoff=0,
                                  anchor='x', side='left', showgrid=True, zeroline=False,
                                  tickfont=dict(family='Myriad Pro',size=16, color='black'), tickcolor='black', 

                                  range=(0, 1000),showline=True, linewidth=1, linecolor='black', mirror=True,
                                  ticks='outside', ticklen=4, tickangle=0))

    fig = go.Figure(data=trace_list, layout=layout)
    return fig

In [68]:
param_space = np.logspace(-3,3,30)
# param_space = np.append(np.geomspace(0.1,92,16), (481.5+92-(np.geomspace(481.5,92,15))[1:]))

In [74]:
num_cores = multiprocessing.cpu_count()

start_time = time.time()

results_list_parallel = Parallel(n_jobs=num_cores, verbose=5)(delayed(obtain_switch_times)(results_list,kdeg_1, kdeg_2)
                                                             for kdeg_1 in param_space
                                                             for kdeg_2 in param_space)

end_time = time.time()    
print('Parallelized Time: ', end_time-start_time, ' seconds')

[Parallel(n_jobs=88)]: Using backend LokyBackend with 88 concurrent workers.
[Parallel(n_jobs=88)]: Done 112 tasks      | elapsed:  1.1min
[Parallel(n_jobs=88)]: Done 274 tasks      | elapsed:  2.2min
[Parallel(n_jobs=88)]: Done 472 tasks      | elapsed:  3.4min
[Parallel(n_jobs=88)]: Done 706 tasks      | elapsed:  4.5min


Parallelized Time:  328.1636264324188  seconds


[Parallel(n_jobs=88)]: Done 900 out of 900 | elapsed:  5.5min finished


In [77]:
len(results_list_parallel)

900